In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import time 
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.models import Sequential, load_model
from keras.layers import LSTM, Dense
from tensorflow import keras
import tensorflow as tf
import csv
from sklearn.decomposition import PCA
import sklearn
import pickle
from keras.layers import LSTM, Dense, SimpleRNN
import matplotlib.pyplot as plt
import statistics
from sklearn.model_selection import train_test_split
from keras.layers import Dense,Dropout,Conv3D,Conv1D,Conv2D,Flatten,MaxPool2D
from sklearn.metrics import accuracy_score
%load_ext tensorboard

In [2]:
import glob
files = glob.glob("../../datasets/camel-1.4.csv", recursive = True)
ant_master = pd.concat(map(pd.read_csv, files))

In [3]:
ant_master

name  version                                             name.1  wmc  \
0    camel      1.4            org.apache.camel.view.RouteDotGenerator    8   
1    camel      1.4  org.apache.camel.component.cxf.feature.Message...    5   
2    camel      1.4             org.apache.camel.builder.script.Groovy    1   
3    camel      1.4         org.apache.camel.component.jhc.JhcProducer   13   
4    camel      1.4      org.apache.camel.bam.model.ActivityDefinition    7   
..     ...      ...                                                ...  ...   
867  camel      1.4  org.apache.camel.component.mail.MailConfiguration   47   
868  camel      1.4     org.apache.camel.util.ExpressionListComparator    3   
869  camel      1.4      org.apache.camel.model.language.PhpExpression    3   
870  camel      1.4         org.apache.camel.util.IntrospectionSupport   20   
871  camel      1.4   org.apache.camel.FailedToCreateProducerException    2   

     dit  noc  cbo  rfc  lcom  ca  ...       dam  moa       mfa       cam  ic  \
0      2    0    8   31    28   1  ...  0.000000    0  0.588235  0.347222   1   
1      0    0    9   17    10   2  ...  0.000000    0  0.000000  0.450000   0   
2      1    0    0    1     0   0  ...  0.000000    0  0.000000  1.000000   0   
3      3    0   41   73    50   5  ...  0.714286    0  0.645161  0.250000   1   
4      2    0    6   13    17   4  ...  1.000000    1  0.333333  0.428571   0   
..   ...  ...  ...  ...   ...  ..  ...       ...  ...       ...       ...  ..   
867    1    0    7   81   829   5  ...  0.857143    0  0.000000  0.183511   0   
868    1    0    4    9     1   1  ...  1.000000    0  0.000000  0.500000   0   
869    2    0    1    5     3   0  ...  0.000000    0  0.956522  0.666667   1   
870    1    0   18   86   188  14  ...  1.000000    0  0.000000  0.276316   0   
871    5    0    4    7     0   2  ...  1.000000    1  0.944444  0.666667   0   

     cbm        amc  max_cc  avg_cc  bug  
0      1  48.250000       7  3.1250    1  
1      0  18.400000       2  1.0000    0  
2      0   0.000000       1  1.0000    0  
3      1  24.923077       5  1.6154    0  
4      0   5.285714       1  0.8571    0  
..   ...        ...     ...     ...  ...  
867    0  13.446809       9  1.4468    0  
868    0  14.333333       2  1.0000    0  
869    2   3.000000       1  0.3333    0  
870    0  35.900000      11  2.5000    1  
871    0  11.000000       1  0.5000    0  

[872 rows x 24 columns]

In [4]:
train, test = train_test_split(ant_master, test_size=0.2)

In [5]:
cols_to_norm = ['wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom', 'ca', 'ce', 'npm', 'lcom3', 'loc', 'dam', 'moa', 'mfa', 'cam', 'ic', 'cbm', 'amc', 'max_cc', 'avg_cc']
x_train = train[cols_to_norm]
x_test = test[cols_to_norm]

In [6]:
x_train

wmc  dit  noc  cbo  rfc  lcom   ca  ce  npm     lcom3  loc  dam  moa  \
36     2    5    0    3    7     0    1   2    2  0.000000   21  1.0    0   
551    8    0    2    8   22    26    3   5    1  0.857143  108  1.0    0   
94     4    5    0    2   11     0    1   1    4  0.500000   52  1.0    0   
509    2    4    0    5    6     0    5   0    2  0.000000   20  1.0    0   
740    5    3    0    9    9     4    2   9    2  0.750000   36  1.0    1   
..   ...  ...  ...  ...  ...   ...  ...  ..  ...       ...  ...  ...  ...   
604   13    2    0   16   27    36    9  11   10  0.880952  119  1.0    3   
501    4    3    4   10   14     0    4   6    3  0.500000   52  1.0    0   
724   25    1    0  149   25   300  141  12   25  2.000000   25  0.0    0   
254    1    1    0  135    1     0  134   1    1  2.000000    1  0.0    0   
807    5    3    0    4   10    10    1   4    3  2.000000   31  0.0    0   

          mfa       cam  ic  cbm        amc  max_cc  avg_cc  
36   0.950000  0.666667   0    0   9.000000       1  0.5000  
551  0.000000  0.400000   0    0  12.375000       5  1.8750  
94   0.850000  0.500000   0    0  11.500000       2  1.0000  
509  0.944444  0.750000   0    0   8.500000       1  0.5000  
740  0.869565  0.300000   0    0   5.600000       1  0.6000  
..        ...       ...  ..  ...        ...     ...     ...  
604  0.541667  0.185185   1    1   7.615385       2  0.9231  
501  0.918919  0.500000   0    0  11.500000       2  1.0000  
724  0.000000  0.180000   0    0   0.000000       1  1.0000  
254  0.000000  1.000000   0    0   0.000000       1  1.0000  
807  0.846154  0.466667   1    1   5.200000       1  0.8000  

[697 rows x 20 columns]

In [7]:
x_test

wmc  dit  noc  cbo  rfc  lcom   ca  ce  npm  lcom3  loc  dam  moa  \
538    6    1    0    2   15    15    0   2    5  2.000   44  0.0    0   
326    2    1    0    2   19     1    1   1    2  2.000  121  0.0    0   
30    11    4    0    8   23    51    1   8   11  0.600   96  1.0    0   
69     9    4    0    9   23     0    3   7    9  0.250   94  1.0    0   
4      7    2    0    6   13    17    4   2    7  0.750   46  1.0    1   
..   ...  ...  ...  ...  ...   ...  ...  ..  ...    ...  ...  ...  ...   
75     5    3    0   18   47     2    2  18    3  0.500  172  1.0    1   
24     3    2    0    4    7     0    1   3    2  0.000   27  1.0    0   
493   26    1    0  389   26   325  387   4   26  2.000   26  0.0    0   
715    9    3    0    8   24    16    3   8    7  0.125  108  1.0    1   
194    3    1    0   63    3     3   62   2    3  2.000    3  0.0    0   

          mfa       cam  ic  cbm        amc  max_cc  avg_cc  
538  0.000000  0.333333   0    0   6.333333       1  0.8333  
326  0.000000  0.666667   0    0  59.500000      13  6.5000  
30   0.957576  0.345455   2    3   7.545455       1  0.6364  
69   0.969325  0.277778   2    3   9.333333       1  0.5556  
4    0.333333  0.428571   0    0   5.285714       1  0.8571  
..        ...       ...  ..  ...        ...     ...     ...  
75   0.846154  0.466667   1    1  33.000000       1  0.8000  
24   0.928571  0.666667   1    1   7.666667       1  0.3333  
493  0.000000  0.157692   0    0   0.000000       1  1.0000  
715  0.850000  0.333333   2    3  10.888889       1  0.6667  
194  0.000000  0.555556   0    0   0.000000       1  1.0000  

[175 rows x 20 columns]

In [8]:
y_train = pd.DataFrame(train['bug'])
y_test = pd.DataFrame(test['bug'])

In [9]:
y_train

bug
36     0
551    0
94     0
509    0
740    0
..   ...
604    0
501    0
724    6
254    0
807    0

[697 rows x 1 columns]

In [10]:
y_test

bug
538    0
326    0
30     1
69     3
4      0
..   ...
75     0
24     0
493    4
715    0
194    0

[175 rows x 1 columns]

In [11]:
#Getting the rows and columns size in our data
img_rows, img_cols = 1,20
# Preparing the data for the model.
X_train_matrix = x_train.values
X_test_matrix = x_test.values
Y_train_matrix = y_train.values
Y_test_matrix = y_test.values
# Preparing the data for the model.
Ytrainseries = y_train['bug']
Ytestseries = y_test['bug']

In [12]:
# Preparing the data for the model.
X_train1 = X_train_matrix.reshape(X_train_matrix.shape[0], img_rows, img_cols, 1)
X_test1 = X_test_matrix.reshape(X_test_matrix.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

In [13]:
def define_model():
    # Designing and initializing the model.
    #Building the model
    model = Sequential()
    #add model layers
    model.add(Conv1D(2, kernel_size=2, activation='relu'))
    model.add(tf.keras.layers.Flatten())
    model.add(Dense(1, activation='relu'))
    #compile model using mse as the loss function
    model.compile(loss='mse', optimizer = keras.optimizers.Adam(learning_rate=0.0005), metrics=['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()])
    return model

In [14]:
y = y_test['bug'].tolist()
y

[0,
 0,
 1,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 2,
 0,
 1,
 2,
 0,
 0,
 2,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 10,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 4,
 0,
 0]

In [15]:
total_training_time = []
total_scores = []
total_training_loss_history = []
test_ARE = []
for i in range(30):
    print("Run no: " + str(i))
    model = define_model()
    log_dir = '/home/bavanya/Documents/Training_curves/cnn_camel1.4/' + 'run' + str(i)
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    start = time.time()
    history = model.fit(X_train1, Y_train_matrix, epochs = 100, batch_size = 100, callbacks=[tensorboard_callback])
    stop = time.time()
    total_training_time.append(stop - start) 
    total_training_loss_history.append(history.history)
    score = model.evaluate(X_test1, Y_test_matrix)
    total_scores.append(score) 
    predictions = model.predict(X_test1)
    x = predictions.tolist()
    x = list(np.concatenate(x))
    x = [int(i) for i in x]
    ARE_test = tf.keras.metrics.MeanRelativeError(normalizer=y)
    ARE_test.update_state(y, x)
    test_ARE.append(ARE_test.result().numpy())
    print("score of " + str(i) + "th run is: " + ', '.join(map(str,score)))

Run no: 0
Epoch 1/100
7/7 [==============================] - 1s 10ms/step - loss: 739.4241 - mse: 739.4241 - mae: 17.1143 - root_mean_squared_error: 27.1924 - mean_squared_logarithmic_error: 5.7832
Epoch 2/100
7/7 [==============================] - 0s 4ms/step - loss: 672.1090 - mse: 672.1090 - mae: 16.3389 - root_mean_squared_error: 25.9251 - mean_squared_logarithmic_error: 5.6187
Epoch 3/100
7/7 [==============================] - 0s 4ms/step - loss: 612.4707 - mse: 612.4707 - mae: 15.6379 - root_mean_squared_error: 24.7481 - mean_squared_logarithmic_error: 5.4681
Epoch 4/100
7/7 [==============================] - 0s 4ms/step - loss: 558.9265 - mse: 558.9265 - mae: 14.9741 - root_mean_squared_error: 23.6416 - mean_squared_logarithmic_error: 5.3124
Epoch 5/100
7/7 [==============================] - 0s 4ms/step - loss: 513.8647 - mse: 513.8647 - mae: 14.3594 - root_mean_squared_error: 22.6686 - mean_squared_logarithmic_error: 5.1656
Epoch 6/100
7/7 [==============================] - 0s 

7/7 [==============================] - 0s 3ms/step - loss: 5.5422 - mse: 5.5422 - mae: 1.0841 - root_mean_squared_error: 2.3542 - mean_squared_logarithmic_error: 0.5624
Epoch 89/100
7/7 [==============================] - 0s 4ms/step - loss: 5.3657 - mse: 5.3657 - mae: 1.0607 - root_mean_squared_error: 2.3164 - mean_squared_logarithmic_error: 0.5500
Epoch 90/100
7/7 [==============================] - 0s 4ms/step - loss: 5.1847 - mse: 5.1847 - mae: 1.0374 - root_mean_squared_error: 2.2770 - mean_squared_logarithmic_error: 0.5379
Epoch 91/100
7/7 [==============================] - 0s 4ms/step - loss: 5.0251 - mse: 5.0251 - mae: 1.0168 - root_mean_squared_error: 2.2417 - mean_squared_logarithmic_error: 0.5271
Epoch 92/100
7/7 [==============================] - 0s 4ms/step - loss: 4.8677 - mse: 4.8677 - mae: 0.9955 - root_mean_squared_error: 2.2063 - mean_squared_logarithmic_error: 0.5154
Epoch 93/100
7/7 [==============================] - 0s 3ms/step - loss: 4.7155 - mse: 4.7155 - mae: 0.9

7/7 [==============================] - 0s 3ms/step - loss: 2.1105 - mse: 2.1105 - mae: 0.4327 - root_mean_squared_error: 1.4528 - mean_squared_logarithmic_error: 0.2521
Epoch 76/100
7/7 [==============================] - 0s 3ms/step - loss: 2.0919 - mse: 2.0919 - mae: 0.4315 - root_mean_squared_error: 1.4463 - mean_squared_logarithmic_error: 0.2516
Epoch 77/100
7/7 [==============================] - 0s 3ms/step - loss: 2.0721 - mse: 2.0721 - mae: 0.4301 - root_mean_squared_error: 1.4395 - mean_squared_logarithmic_error: 0.2510
Epoch 78/100
7/7 [==============================] - 0s 3ms/step - loss: 2.0554 - mse: 2.0554 - mae: 0.4291 - root_mean_squared_error: 1.4337 - mean_squared_logarithmic_error: 0.2505
Epoch 79/100
7/7 [==============================] - 0s 3ms/step - loss: 2.0399 - mse: 2.0399 - mae: 0.4280 - root_mean_squared_error: 1.4283 - mean_squared_logarithmic_error: 0.2500
Epoch 80/100
7/7 [==============================] - 0s 3ms/step - loss: 2.0324 - mse: 2.0324 - mae: 0.4

7/7 [==============================] - 0s 4ms/step - loss: 1.8862 - mse: 1.8862 - mae: 0.4108 - root_mean_squared_error: 1.3734 - mean_squared_logarithmic_error: 0.2385
Epoch 63/100
7/7 [==============================] - 0s 3ms/step - loss: 1.8862 - mse: 1.8862 - mae: 0.4109 - root_mean_squared_error: 1.3734 - mean_squared_logarithmic_error: 0.2385
Epoch 64/100
7/7 [==============================] - 0s 4ms/step - loss: 1.8862 - mse: 1.8862 - mae: 0.4109 - root_mean_squared_error: 1.3734 - mean_squared_logarithmic_error: 0.2385
Epoch 65/100
7/7 [==============================] - 0s 4ms/step - loss: 1.8862 - mse: 1.8862 - mae: 0.4109 - root_mean_squared_error: 1.3734 - mean_squared_logarithmic_error: 0.2385
Epoch 66/100
7/7 [==============================] - 0s 3ms/step - loss: 1.8862 - mse: 1.8862 - mae: 0.4109 - root_mean_squared_error: 1.3734 - mean_squared_logarithmic_error: 0.2385
Epoch 67/100
7/7 [==============================] - 0s 4ms/step - loss: 1.8862 - mse: 1.8862 - mae: 0.4

7/7 [==============================] - 0s 3ms/step - loss: 1.8873 - mse: 1.8873 - mae: 0.4110 - root_mean_squared_error: 1.3738 - mean_squared_logarithmic_error: 0.2391
Epoch 50/100
7/7 [==============================] - 0s 3ms/step - loss: 1.8873 - mse: 1.8873 - mae: 0.4109 - root_mean_squared_error: 1.3738 - mean_squared_logarithmic_error: 0.2391
Epoch 51/100
7/7 [==============================] - 0s 3ms/step - loss: 1.8872 - mse: 1.8872 - mae: 0.4108 - root_mean_squared_error: 1.3738 - mean_squared_logarithmic_error: 0.2390
Epoch 52/100
7/7 [==============================] - 0s 3ms/step - loss: 1.8872 - mse: 1.8872 - mae: 0.4108 - root_mean_squared_error: 1.3738 - mean_squared_logarithmic_error: 0.2390
Epoch 53/100
7/7 [==============================] - 0s 3ms/step - loss: 1.8872 - mse: 1.8872 - mae: 0.4107 - root_mean_squared_error: 1.3738 - mean_squared_logarithmic_error: 0.2390
Epoch 54/100
7/7 [==============================] - 0s 3ms/step - loss: 1.8872 - mse: 1.8872 - mae: 0.4

...

7/7 [==============================] - 0s 4ms/step - loss: 1.8867 - mse: 1.8867 - mae: 0.4089 - root_mean_squared_error: 1.3736 - mean_squared_logarithmic_error: 0.2386
Epoch 51/100
7/7 [==============================] - 0s 4ms/step - loss: 1.8867 - mse: 1.8867 - mae: 0.4089 - root_mean_squared_error: 1.3736 - mean_squared_logarithmic_error: 0.2386
Epoch 52/100
7/7 [==============================] - 0s 5ms/step - loss: 1.8867 - mse: 1.8867 - mae: 0.4089 - root_mean_squared_error: 1.3736 - mean_squared_logarithmic_error: 0.2386
Epoch 53/100
7/7 [==============================] - 0s 4ms/step - loss: 1.8867 - mse: 1.8867 - mae: 0.4089 - root_mean_squared_error: 1.3736 - mean_squared_logarithmic_error: 0.2386
Epoch 54/100
7/7 [==============================] - 0s 5ms/step - loss: 1.8867 - mse: 1.8867 - mae: 0.4089 - root_mean_squared_error: 1.3736 - mean_squared_logarithmic_error: 0.2386
Epoch 55/100
7/7 [==============================] - 0s 5ms/step - loss: 1.8867 - mse: 1.8867 - mae: 0.4

7/7 [==============================] - 0s 5ms/step - loss: 2449.9104 - mse: 2449.9104 - mae: 4.8219 - root_mean_squared_error: 49.4966 - mean_squared_logarithmic_error: 0.7867
Epoch 37/100
7/7 [==============================] - 0s 4ms/step - loss: 2401.1875 - mse: 2401.1875 - mae: 4.7468 - root_mean_squared_error: 49.0019 - mean_squared_logarithmic_error: 0.7757
Epoch 38/100
7/7 [==============================] - 0s 5ms/step - loss: 2356.4941 - mse: 2356.4941 - mae: 4.6732 - root_mean_squared_error: 48.5437 - mean_squared_logarithmic_error: 0.7647
Epoch 39/100
7/7 [==============================] - 0s 4ms/step - loss: 2288.1467 - mse: 2288.1467 - mae: 4.5890 - root_mean_squared_error: 47.8346 - mean_squared_logarithmic_error: 0.7544
Epoch 40/100
7/7 [==============================] - 0s 5ms/step - loss: 2229.7456 - mse: 2229.7456 - mae: 4.5141 - root_mean_squared_error: 47.2202 - mean_squared_logarithmic_error: 0.7457
Epoch 41/100
7/7 [==============================] - 0s 5ms/step - lo

7/7 [==============================] - 0s 4ms/step - loss: 18.3989 - mse: 18.3989 - mae: 1.4315 - root_mean_squared_error: 4.2894 - mean_squared_logarithmic_error: 0.6678
Epoch 20/100
7/7 [==============================] - 0s 4ms/step - loss: 16.1973 - mse: 16.1973 - mae: 1.3272 - root_mean_squared_error: 4.0246 - mean_squared_logarithmic_error: 0.6251
Epoch 21/100
7/7 [==============================] - 0s 5ms/step - loss: 14.1645 - mse: 14.1645 - mae: 1.2289 - root_mean_squared_error: 3.7636 - mean_squared_logarithmic_error: 0.5861
Epoch 22/100
7/7 [==============================] - 0s 3ms/step - loss: 12.5457 - mse: 12.5457 - mae: 1.1451 - root_mean_squared_error: 3.5420 - mean_squared_logarithmic_error: 0.5551
Epoch 23/100
7/7 [==============================] - 0s 4ms/step - loss: 11.0288 - mse: 11.0288 - mae: 1.0645 - root_mean_squared_error: 3.3210 - mean_squared_logarithmic_error: 0.5211
Epoch 24/100
7/7 [==============================] - 0s 4ms/step - loss: 9.8722 - mse: 9.8722 

7/7 [==============================] - 0s 4ms/step - loss: 2.1256 - mse: 2.1256 - mae: 0.4388 - root_mean_squared_error: 1.4580 - mean_squared_logarithmic_error: 0.2449
Epoch 65/100
7/7 [==============================] - 0s 4ms/step - loss: 2.1138 - mse: 2.1138 - mae: 0.4376 - root_mean_squared_error: 1.4539 - mean_squared_logarithmic_error: 0.2443
Epoch 66/100
7/7 [==============================] - 0s 5ms/step - loss: 2.1049 - mse: 2.1049 - mae: 0.4362 - root_mean_squared_error: 1.4508 - mean_squared_logarithmic_error: 0.2434
Epoch 67/100
7/7 [==============================] - 0s 5ms/step - loss: 2.0930 - mse: 2.0930 - mae: 0.4348 - root_mean_squared_error: 1.4467 - mean_squared_logarithmic_error: 0.2425
Epoch 68/100
7/7 [==============================] - 0s 4ms/step - loss: 2.0859 - mse: 2.0859 - mae: 0.4336 - root_mean_squared_error: 1.4443 - mean_squared_logarithmic_error: 0.2418
Epoch 69/100
7/7 [==============================] - 0s 4ms/step - loss: 2.0757 - mse: 2.0757 - mae: 0.4

7/7 [==============================] - 0s 4ms/step - loss: 1.8867 - mse: 1.8867 - mae: 0.4092 - root_mean_squared_error: 1.3736 - mean_squared_logarithmic_error: 0.2386
Epoch 52/100
7/7 [==============================] - 0s 5ms/step - loss: 1.8867 - mse: 1.8867 - mae: 0.4092 - root_mean_squared_error: 1.3736 - mean_squared_logarithmic_error: 0.2386
Epoch 53/100
7/7 [==============================] - 0s 5ms/step - loss: 1.8867 - mse: 1.8867 - mae: 0.4092 - root_mean_squared_error: 1.3736 - mean_squared_logarithmic_error: 0.2386
Epoch 54/100
7/7 [==============================] - 0s 5ms/step - loss: 1.8867 - mse: 1.8867 - mae: 0.4092 - root_mean_squared_error: 1.3736 - mean_squared_logarithmic_error: 0.2386
Epoch 55/100
7/7 [==============================] - 0s 4ms/step - loss: 1.8867 - mse: 1.8867 - mae: 0.4091 - root_mean_squared_error: 1.3736 - mean_squared_logarithmic_error: 0.2386
Epoch 56/100
7/7 [==============================] - 0s 4ms/step - loss: 1.8867 - mse: 1.8867 - mae: 0.4

In [16]:
%tensorboard --logdir '/home/bavanya/Documents/Training_curves/cnn_camel1.4/'

In [17]:
total_training_time

[4.793623447418213,
 4.436676025390625,
 4.423286437988281,
 4.173327207565308,
 4.11555552482605,
 4.157557725906372,
 4.744056940078735,
 5.118350028991699,
 5.064071178436279,
 5.1136860847473145,
 4.976345062255859,
 4.435213565826416,
 4.869687080383301,
 4.692361354827881,
 5.224011659622192,
 5.290436029434204,
 5.241187572479248,
 4.354648113250732,
 3.900249481201172,
 4.237276554107666,
 5.917529344558716,
 6.125537633895874,
 6.2711405754089355,
 6.012564659118652,
 6.080844879150391,
 5.948365926742554,
 5.927888631820679,
 5.9376466274261475,
 5.693885803222656,
 5.846611022949219]

In [18]:
print("avg training time: " + str(sum(total_training_time)/30))

avg training time: 5.104120739301046


In [19]:
total_training_loss_history

[{'loss': [739.424072265625,
   672.1090087890625,
   612.470703125,
   558.9264526367188,
   513.86474609375,
   469.7206726074219,
   429.6196594238281,
   394.34326171875,
   362.11431884765625,
   334.3852844238281,
   306.2293395996094,
   281.89105224609375,
   260.5546569824219,
   242.14669799804688,
   224.0540313720703,
   208.74365234375,
   192.93597412109375,
   179.9060516357422,
   168.31871032714844,
   157.02928161621094,
   147.26199340820312,
   138.01043701171875,
   129.20086669921875,
   121.70663452148438,
   114.28176879882812,
   107.60260772705078,
   101.41423034667969,
   95.66084289550781,
   90.07105255126953,
   85.16128540039062,
   80.32978820800781,
   75.6621322631836,
   71.507080078125,
   67.55326843261719,
   63.92085647583008,
   60.31671905517578,
   57.088844299316406,
   53.99494552612305,
   51.1044807434082,
   48.37025833129883,
   45.892494201660156,
   43.500919342041016,
   41.3049201965332,
   39.20746612548828,
   37.20204162597656,
  

In [20]:
sum_mse = 0
sum_mae = 0
training_mse = []
training_mae = []
for i in range(30):
    sum_mse += float(total_training_loss_history[i]['mse'][-1])
    sum_mae += float(total_training_loss_history[i]['mae'][-1])
    training_mse.append(total_training_loss_history[i]['mse'][-1])
    training_mae.append(total_training_loss_history[i]['mae'][-1])
print("avg mse of training: " + str(sum_mse/30))
print("avg mae of training: " + str(sum_mae/30))

avg mse of training: 39.72971756458283
avg mae of training: 0.5580654164155324


In [21]:
total_scores

[[2.4644851684570312,
  2.4644851684570312,
  0.7241150140762329,
  1.5698678493499756,
  0.3896868824958801],
 [1.0742857456207275,
  1.0742857456207275,
  0.2857142984867096,
  1.0364775657653809,
  0.16591180860996246],
 [1.0746902227401733,
  1.0746902227401733,
  0.288347452878952,
  1.0366727113723755,
  0.16626182198524475],
 [1.0743613243103027,
  1.0743613243103027,
  0.286485880613327,
  1.0365140438079834,
  0.16597974300384521],
 [1.0742857456207275,
  1.0742857456207275,
  0.2857142984867096,
  1.0364775657653809,
  0.16591180860996246],
 [1.1746608018875122,
  1.1746608018875122,
  0.32969194650650024,
  1.083817720413208,
  0.1921517252922058],
 [1.489980697631836,
  1.489980697631836,
  0.4361746609210968,
  1.2206476926803589,
  0.2520250082015991],
 [1.1638001203536987,
  1.1638001203536987,
  0.36272644996643066,
  1.0787956714630127,
  0.1793215423822403],
 [1.214755654335022,
  1.214755654335022,
  0.3140459656715393,
  1.1021595001220703,
  0.18411357700824738],
 

In [4]:
a = [[2.4644851684570312,
  2.4644851684570312,
  0.7241150140762329,
  1.5698678493499756,
  0.3896868824958801],
 [1.0742857456207275,
  1.0742857456207275,
  0.2857142984867096,
  1.0364775657653809,
  0.16591180860996246],
 [1.0746902227401733,
  1.0746902227401733,
  0.288347452878952,
  1.0366727113723755,
  0.16626182198524475],
 [1.0743613243103027,
  1.0743613243103027,
  0.286485880613327,
  1.0365140438079834,
  0.16597974300384521],
 [1.0742857456207275,
  1.0742857456207275,
  0.2857142984867096,
  1.0364775657653809,
  0.16591180860996246],
 [1.1746608018875122,
  1.1746608018875122,
  0.32969194650650024,
  1.083817720413208,
  0.1921517252922058],
 [1.489980697631836,
  1.489980697631836,
  0.4361746609210968,
  1.2206476926803589,
  0.2520250082015991],
 [1.1638001203536987,
  1.1638001203536987,
  0.36272644996643066,
  1.0787956714630127,
  0.1793215423822403],
 [1.214755654335022,
  1.214755654335022,
  0.3140459656715393,
  1.1021595001220703,
  0.18411357700824738],
 [1.1567283868789673,
  1.1567283868789673,
  0.46970459818840027,
  1.0755131244659424,
  0.20330333709716797],
 [1.0742857456207275,
  1.0742857456207275,
  0.2857142984867096,
  1.0364775657653809,
  0.16591180860996246],
 [1.2163710594177246,
  1.2163710594177246,
  0.4108572006225586,
  1.1028921604156494,
  0.20460721850395203],
 [1.0772814750671387,
  1.0772814750671387,
  0.2943708598613739,
  1.037921667098999,
  0.16802681982517242],
 [1.1690666675567627,
  1.1690666675567627,
  0.3285650610923767,
  1.0812338590621948,
  0.19131219387054443],
 [1.0742857456207275,
  1.0742857456207275,
  0.2857142984867096,
  1.0364775657653809,
  0.16591180860996246],
 [1.0742857456207275,
  1.0742857456207275,
  0.2857142984867096,
  1.0364775657653809,
  0.16591180860996246],
 [1.0742857456207275,
  1.0742857456207275,
  0.2857142984867096,
  1.0364775657653809,
  0.16591180860996246],
 [1.0742857456207275,
  1.0742857456207275,
  0.2857142984867096,
  1.0364775657653809,
  0.16591180860996246],
 [1.0742857456207275,
  1.0742857456207275,
  0.2857142984867096,
  1.0364775657653809,
  0.16591180860996246],
 [2.5093512535095215,
  2.5093512535095215,
  0.3851727247238159,
  1.5840932130813599,
  0.17997044324874878],
 [1.7851898670196533,
  1.7851898670196533,
  0.41444846987724304,
  1.33610999584198,
  0.21626068651676178],
 [1.213757872581482,
  1.213757872581482,
  0.3496762216091156,
  1.1017067432403564,
  0.1991887390613556],
 [0.9867013096809387,
  0.9867013096809387,
  0.2675440311431885,
  0.9933283925056458,
  0.15124200284481049],
 [1.7183705568313599,
  1.7183705568313599,
  0.5087791681289673,
  1.310866355895996,
  0.2574433982372284],
 [1.0742857456207275,
  1.0742857456207275,
  0.2857142984867096,
  1.0364775657653809,
  0.16591180860996246],
 [1.0742857456207275,
  1.0742857456207275,
  0.2857142984867096,
  1.0364775657653809,
  0.16591180860996246],
 [1.0742857456207275,
  1.0742857456207275,
  0.2857142984867096,
  1.0364775657653809,
  0.16591180860996246],
 [56.029056549072266,
  56.029056549072266,
  1.041699767112732,
  7.485255718231201,
  0.3377349376678467],
 [1.1870626211166382,
  1.1870626211166382,
  0.3188765048980713,
  1.0895240306854248,
  0.18561549484729767],
 [1.0742857456207275,
  1.0742857456207275,
  0.2857152223587036,
  1.0364775657653809,
  0.16591180860996246]]

In [5]:
test_mae = []
for i in range(10):
    test_mae.append(a[i][2])

In [6]:
test_mae

[0.7241150140762329,
 0.2857142984867096,
 0.288347452878952,
 0.286485880613327,
 0.2857142984867096,
 0.32969194650650024,
 0.4361746609210968,
 0.36272644996643066,
 0.3140459656715393,
 0.46970459818840027]

In [2]:
t = [0.5488777756690979,
 0.49643194675445557,
 0.4505343735218048,
 0.5004401803016663,
 0.46572577953338623,
 0.4550250768661499,
 0.5667808055877686,
 0.5136887431144714,
 0.5308082699775696,
 0.5325367450714111,
 0.42035597562789917,
 0.5127224326133728,
 0.46964049339294434,
 0.4734709560871124,
 0.5981016159057617,
 0.49342936277389526,
 0.5519396662712097,
 0.46647509932518005,
 0.4910261929035187,
 0.569240391254425,
 0.4442990720272064,
 0.4285086393356323,
 0.4999818205833435,
 0.5248304009437561,
 0.474327951669693,
 0.6028088331222534,
 0.48042967915534973,
 0.46979865431785583,
 0.5090309977531433,
 0.46979865431785583]
print(sum(t[:10])/10)

0.5060849696397781


In [25]:
sum_mse = 0
sum_mae = 0
sum_are = 0
test_mse = []
test_mae = []
for i in range(30):
    sum_mse += float(total_scores[i][0])
    sum_mae += float(total_scores[i][2])
    test_mse.append(total_scores[i][0])
    test_mae.append(total_scores[i][2])
    sum_are += float(test_ARE[i])
print("avg mse: " + str(sum_mse/30))
print("avg mae: " + str(sum_mae/30))
print("avg are: " + str(sum_are/30))

avg mse: 3.086570018529892
avg mae: 0.36532848278681435
avg are: 0.14087301244338354


In [26]:
print("median training mse: " + str(statistics.median(training_mse)))
print("median testing mse: " + str(statistics.median(test_mse)))

median training mse: 1.8881757259368896
median testing mse: 1.075985848903656


In [27]:
print("median training mae: " + str(statistics.median(training_mae)))
print("median testing mae: " + str(statistics.median(test_mae)))

median training mae: 0.42271673679351807
median testing mae: 0.29135915637016296


In [28]:
print("median testing ARE: " + str(statistics.median(test_ARE)))

median testing ARE: 0.1371428519487381


In [29]:
test_ARE

[0.110476196,
 0.13714285,
 0.13714285,
 0.13714285,
 0.13714285,
 0.13714285,
 0.13714285,
 0.13714285,
 0.13714285,
 0.13714285,
 0.13714285,
 0.13428572,
 0.13714285,
 0.13714285,
 0.13714285,
 0.13714285,
 0.13714285,
 0.13714285,
 0.13714285,
 0.15428571,
 0.14428571,
 0.13714285,
 0.13142857,
 0.12857144,
 0.13714285,
 0.13714285,
 0.13714285,
 0.26857144,
 0.13714285,
 0.13714285]

In [1]:
t = [0.110476196,
 0.13714285,
 0.13714285,
 0.13714285,
 0.13714285,
 0.13714285,
 0.13714285,
 0.13714285,
 0.13714285,
 0.13714285,
 0.13714285,
 0.13428572,
 0.13714285,
 0.13714285,
 0.13714285,
 0.13714285,
 0.13714285,
 0.13714285,
 0.13714285,
 0.15428571,
 0.14428571,
 0.13714285,
 0.13142857,
 0.12857144,
 0.13714285,
 0.13714285,
 0.13714285,
 0.26857144,
 0.13714285,
 0.13714285]
print(sum(t[:10])/10)

0.13447618460000002
